## Extract and Summarize Nanoindentation Data

This notebook is used for extraction of specified nanoindentation data from .xlsx files from the UKEA. The notebook will selectively extract data from a range of files, by taking only the data from a requested depth. Given a list of these requested depths, the code will take the datapoint closest to each depth. This will be done for each nanoindentation point in the data. This will yield a dataframe containing all load vs depth data relevant to these depths, which can either be used by the user for direct, analysis, or can also be recorded in a pre-prepared excel summary sheet.

In [ ]:
import os
import re
import fnmatch
import pandas as pd
import pylightxl as xl

#### Parameters:
- **directory** = full path to folder containing all data files. The script will automatically recognise linescan files among others, as long as they follow the filename convention "line_scan#.xlsx" where # is an integer indicating linescan number (between 0-9). Path should prefarably be a [Raw String](https://docs.python.org/3/reference/lexical_analysis.html), especially if running on windows os. 

- **SUMMARY_FILE** = string path to summary file. 

- **DEPTHS** = list of all desired depths at which to get load data. The code will look to the closest value around each depth and extract the corresponding load value

In [ ]:

directory = r"C:\Users\robya\Documents\SampleData"
SUMMARY_FILE = f"C:\\Users\\robya\\Documents\\Summer Internship\\OneDrive_1_16-08-2022\\E97 Data set 1\\Time-resolved summary.xlsx"
DEPTHS = [200, 300]


#### Function Definitions:

In [ ]:
def get_test_array(file, depths=[200]):
    """
    Produces a list of pandas dataframes containing load vs depth data for for all tests from a given file
    IN: -file = string containing full file path, preferably a raw sting
        -depths = list containing all desired depts at which to gain data.
        -num_of_indentations = the amount of tests expected in the sheet. Default set to 40. If less are found, function will print a warning
    OUT:-list of dataframes for each supplied depth
    """
    
    def get_depth_load_data(file, depth):
        """ Produce a dataframe of all loads within this linescan file at given depth"""
        
        depth_data = pd.DataFrame(columns = ["Depth [nm]", "Load [mN]"])
        for num in range(1,100):   
            """Loops between all test numbers enclosed within the range object"""
            try:
                num = str(num).zfill(3)                                                        # padds the number with zeros so it is formated as "0XX" 
                data = pd.read_excel(file, sheet_name=f"Test {num}")
                df = pd.to_numeric(data['Displacement Into Surface'][1:].sub(depth).abs())     # obtain the value closest to current selected depth NOTE: This is not the fastest way to do it
                idx = df.idxmin()                                                              # index at which closest value occurs
                datapoint = [data["Displacement Into Surface"].iloc[idx], data["Load On Sample"].iloc[idx]]# displacement and idx datapoint                              
                depth_data = depth_data.append(pd.DataFrame([datapoint],  #append datapoint to depth_data 
                     columns=["Depth [nm]", "Load [mN]"]), 
                     ignore_index=True)        

            except ValueError:
                print(f"A total of {int(num)-1} Nanoindentation points were found @ depth of {depth}nm")
                print("_______________________________________________________________________________")
                break
        depth_data.index += 1         #Begin index count at 1
        return depth_data
    
    
    listed_data = []
    for depth in depths:
        listed_data.append(get_depth_load_data(file, depth))
    
    print(f"If you expect more, ensure that all sheets in file follow the format [Test XXX] \n and that sequence of numbers is continuous and does not skip any said numbers")
    return listed_data
    
    
def find_linescan_files(directory):
    """
    Find all linescan files in input directory path
    IN: -directory = string with directory name
    OUT:-list with all linescan files in directory with full paths
    """
    files = []
    for file in os.listdir(directory):
        if file.endswith(".xlsx"):
            files.append(os.path.splitext(file)[0])

    files = fnmatch.filter(files, 'line_scan?') 
    new_files = []
    
    for file in files:
        file = os.path.join(directory,file+".xlsx")
        new_files.append(file)

    return new_files


def gather_linescan_data(directory):
    """
    Yields a dataframe containing all data from the linescans from UKEA
    IN: -directory= string with full path to linescan files
    OUT:-dataframe with data from all linescans in folder 
    """
    files = find_linescan_files(directory)
    Linescans = [] 
    for file in files:
        linescan_number = file[-6]           # The linescan number taken from filenames
        print(f"\n\n[Linescan {linescan_number} ]========================================================")
        data = get_test_array(file, DEPTHS)
        data = pd.DataFrame([data], columns=DEPTHS, index=[linescan_number])
        data.index.name = "Linescan No."
        Linescans.append(data)          
    #print(Linescans)
    data = pd.concat(Linescans)
    return data


def summarize_linescan(summary_file, full_data):
    """
    Fill a summary file with data. Will write it into the excel sheet
    IN: -summary_file= string of full path to summary file
        -data= array with na-ion data for all depths as pd dataframes
    OUT:-excel database ready to be saved as a file
    """
    #READ DATABASE
    db = xl.readxl(fn=summary_file)
    for x in full_data.index:
        current_sheet = f"Linescan-{x}"
        data = full_data.loc[x]
        data = [x for x in data]
        
        #Get the number of tests
        num_of_tests = db.ws(ws=current_sheet).index(row=3, col=1)
        for row_to_edit in range(3,num_of_tests+3):  
            col_to_edit = 2
               # for col_to_edit in range(2,4,2):
            #Read index of first column,
            current_idx = db.ws(ws=current_sheet).index(row=row_to_edit, col=1)

            for datafr in data:
                #Fetch index from dataframe
                curr_value = datafr["Load [mN]"].loc[current_idx]#[0]
                #Write value from dataframe into database
                db.ws(ws=current_sheet).update_index(row=+row_to_edit, col=col_to_edit, val=curr_value)
                col_to_edit +=2

    #Repeat for next indentation depth
    #print(summary_file)
    xl.writexl(db=db, fn=summary_file)
    return db



#### Main Cell
Running this cell will execute the whole code. Note that the summarize_linescan function will only input. The "Linescans" variable will yield a DataFrame. Each cell houses data from a given linescan file (index of cell corresponding to the number of the linescan, and the column name of the cell corresponds to the depth at which the data was taken.



![Image1](.\Full_df.png).




Where each cell will house a dataframe containing depth vs load data. Each index corresponds to the indentation number, as specified in the linescan file.


![Image2](.\mini_df.png).


In [ ]:
# Run this line to get a dataframe containing all data from a dataset of xlsx files
Linescans = gather_linescan_data(directory)

# Run this line to input the data into a set SUMMARY_FILE. The output of this function will give you a database object 
# (from pylightxl) for analysis of the output to the summary file, without opening the excel itself
_ = summarize_linescan(SUMMARY_FILE, Linescans)

#### Notes
Note that the summarize_linescan function is not very smart. The user will need to prepare the summary file in an appropriate format. Each Linescan should have an appropriately named sheet using the format "Linescan-#", and should have the number of indentations expected. 

![Image3](.\sum_sheet.png).